In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pybetacool import PyBetaCool

neutralisation_list = [0,0.5,1] # fraction

for i,neutralisation_space_charge_xs in enumerate(neutralisation_list):
    neutralisation_space_charge_betacool=(1-neutralisation_space_charge_xs)*100 #xsuite and betacool use different definitions of neutralisation
    BLDfilename = '/home/pkruyt/cernbox/BETACOOL_ecooling/benchmarking-Betacool/ELENA.bld'
    betacoolExec = '/home/pkruyt/cernbox/BETACOOL_ecooling/betacool/Betacool'
    obj = PyBetaCool(BLDfilename=BLDfilename, betacoolExec=betacoolExec)
    # Edit one parameter (row=50, 4th value is "magnetic field")
    obj.BLDContent[55][3] = neutralisation_space_charge_betacool 
    # Overwrites LEIR.bld with new setting 
    obj.BLDContent.save(BLDfilename)
    # Execute Betacool
    obj.runBetacool('/m')
    tmp = obj.parseCurveFile('emittance.cur')

    time=tmp[:,0]
    emittance_x = tmp[:,1]*1e-6
    emittance_y = tmp[:,2]*1e-6
    dp_p=tmp[:,3]

    filpath=f'results/emittance_betacool_SC={neutralisation_space_charge_xs}.npz'
    #reset the bld file back to default 
    obj.BLDContent[55][3] = 100 
    # Overwrites LEIR.bld with new setting 
    obj.BLDContent.save(BLDfilename)
    
    np.savez(filpath, time=time, emittance_x=emittance_x, emittance_y=emittance_y, dp_p=dp_p)

    # Print progress indication
    progress = (i + 1) / len(neutralisation_list) * 100
    print(f"Progress: {progress:.1f}%")